<a href="https://colab.research.google.com/github/yule0707/Lajittelija/blob/main/Suoritusj%C3%A4rjestys.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Python OOP: Muuttujien näkyvyysalue ja suoritusjärjestys

**Sääntö:** Ennusta aina ennen kuin suoritat koodin.

Täytä joka kohdassa:
- **Ennuste:** mitä uskot tulostuvan / tapahtuvan
- **Selitys:** miksi näin tapahtuu (scope + suoritusjärjestys)


## 1.1 Lokaali vs globaali

**Ennuste (ennen suorittamista):**
- Mitä tulostuu? Miksi?

**Selitys (suorittamisen jälkeen):**
- Mitä näkyvyysaluetta Python käytti?


In [6]:
x = "GLOBAL"

def f():
    x = "LOCAL"
    print("Inside f:", x)

f()
print("Outside:", x)


Inside f: LOCAL
Outside: GLOBAL


Tulostuu
Inside Global
Outside Global

Kosa x=Globaali muuttuja. Ensin suoritetaan funktio, jossa sisäinen muuttuja.

## 1.2 Luku vs sijoitus -sääntö

**Ennuste:**  
- Toimiiko tämä? Jos ei, mikä virhe tulee ja miksi?

**Selitys:**  
- Mitä Python olettaa, kun muuttujalle tehdään sijoitus funktion sisällä?


In [7]:
y = 10

def g():
    # Uncomment the next line and run:
    # print("y is", y)
    y = 99
    print("set y to", y)

g()
print("global y is still", y)


set y to 99
global y is still 10


Tulostuu
 set y to 99
 global y is still 10

 funktio hakee muuttujaa aluksi funktion sisältä ja koska sellainen on muodostetttu, se ei etsi globaalia muuttujaa.

## 1.3 Enclosing-scope (sisäkkäinen funktio)

**Ennuste:**  
- Mitä tulostuu? Miksi?

**Selitys:**  
- Tunnista Local vs Enclosing vs Global.


In [8]:
z = "GLOBAL Z"

def outer():
    z = "ENCLOSING Z"
    def inner():
        print("inner sees:", z)
    inner()

outer()
print("global sees:", z)


inner sees: ENCLOSING Z
global sees: GLOBAL Z


Tulostuu
inner sees ENCLOSING Z
global sees GBLOAL Z

Inner funktio etsii muuttujaa ensin outer funtiosta "enclosing muuttuja". z löytyy, joten inner käyttää outer enclosing muuttujaa.

## 2.1 Luokka-attribuutti vs instanssiattribuutti

**Ennuste:**
- Mitä kukin print-lause tulostaa?

**Selitys:**
- Kun käytetään `obj.value`, mistä Python etsii arvon ensin?


In [9]:
class Counter:
    value = 100  # class attribute

    def __init__(self):
        self.value = 1  # instance attribute

c = Counter()
print("c.value =", c.value)
print("Counter.value =", Counter.value)

# Peek inside instance namespace:
print("c.__dict__ =", c.__dict__)


c.value = 1
Counter.value = 100
c.__dict__ = {'value': 1}


Tuloste:
c.value = 1
Counter.value = 100
c.__dict_-= {value : 1}
Python etsii arvon ensin olion attribuuteista.

## 2.2 Nimen varjostaminen (shadowing)

**Ennuste:**
- Mitä tulostuu ja miksi?

**Selitys:**
- Mitä `name`-muuttujaa käytetään missäkin kohdassa?


In [10]:
class Person:
    def __init__(self, name):
        self.name = name

    def greet(self):
        name = "LOCAL NAME"
        print("local name:", name)
        print("instance name:", self.name)

p = Person("Ada")
p.greet()


local name: LOCAL NAME
instance name: Ada


Tulostuu:
local name LOCAL NAME
Instance name Ada

local name käytetään metodin lokaalia muuttujaa, jota ei voi käyttää metodin ulkopuolella.
self.name käytetään olion obj.value muuttujaa.


## 2.3 Jaettu luokka-attribuutti – yleinen sudenkuoppa

**Ennuste:**
- Mitä ovat `a.tags` ja `b.tags` append-operaation jälkeen?

**Selitys:**
- Miksi tämä voi olla vaarallista?


In [11]:
class Item:
    tags = []  # class attribute (shared!)

a = Item()
b = Item()

a.tags.append("fragile")
print("a.tags:", a.tags)
print("b.tags:", b.tags)  # surprise?


a.tags: ['fragile']
b.tags: ['fragile']


a.tags fragile
b.tags fragile

Jos käytetään globaalia muuttujaa säilyttämään olion arvoja se johtaa odottamattomiin lopputuloksiin.
Virheenkorjaus, vianhaku ja luettavuus hankaloituu.

## 2.4 TODO: Tee tags-listasta instanssikohtainen

Tavoite:
- `a.tags` pitäisi olla `["fragile"]`
- `b.tags` pitäisi olla `[]`

Muokkaa luokkaa niin, että jokaisella oliolla on oma lista.


In [12]:
class ItemFixed:
  def __init__(self):
    self.tags = []

a = ItemFixed()
b = ItemFixed()

a.tags.append("fragile")

print("a.tags:", a.tags)
print("b.tags:", b.tags)

# Self-check (should pass)
assert a.tags == ["fragile"]
assert b.tags == []
print("OK")


a.tags: ['fragile']
b.tags: []
OK


## 3.1 Suoritusjärjestys: luokan runko vs __init__

**Ennuste:**
- Missä järjestyksessä tulosteet tapahtuvat?

**Selitys:**
- Milloin luokan runko suoritetaan?
- Milloin `__init__` suoritetaan?


In [13]:
print("TOP: module start")

class Demo:
    print("A: inside class body (runs at class definition time)")

    def __init__(self):
        print("B: inside __init__ (runs at object creation time)")

    def method(self):
        print("C: inside method (runs when called)")

print("MIDDLE: before creating object")
d = Demo()
print("MIDDLE: before calling method")
d.method()
print("BOTTOM: module end")


TOP: module start
A: inside class body (runs at class definition time)
MIDDLE: before creating object
B: inside __init__ (runs at object creation time)
MIDDLE: before calling method
C: inside method (runs when called)
BOTTOM: module end


TOP
A:
MIDDLE: before creating object
B:
MIDDLE: before calling method
C:
BOTTOM:

Luokan runko suoritetaan heti kun luokka on tehty.

__INIT_ suoritetaan aina kun uusi olio luodaan.

## Pohdittavaa

1) Kuvaile omin sanoin, mikä on hakujärjestys, kun Python evaluoi `obj.attr`.
2) Mikä on ero:
   - luokka-attribuutin
   - instanssiattribuutin
   - metodin sisäisen lokaalin muuttujan välillä?
3) Milloin luokan runko suoritetaan verrattuna `__init__`-metodiin?


1. Kun python evaluoi obj.attr, hakee ensin instanssiattribuuteista.
Seuraavaksi Python etsii sitä luokka-attribuuteista.
Lopuksi Python etsii metodeista.
2. Luokka attribuutti on kaikille metodeille näkyvä ja pysyy muuttumattomana. Sitä käytetään kun halutaan hakea usealle metodille samaa tietoa.
Instanssiattribuutti määritellään yleensä luokan __init__metodissa, esim self.attribuutti = arvo. Se on yksilöllinen jokaiselle oliolle ja on olemassa niin kauan kuin olio on olemassa. Kuvaa yksittäisen olion tilaa ja ominaisuuksia.
Metodin sisäinen muuttuja määritellään metodin sisällä ja on olemassa ja saatavilla vain metodissa jossa se on määritelty. Se luodaan kun metodi kutsutaan ja tuhoutuu sen jälkeen. Käytetään väliaikaisille arvoille joita käytetään vain metodin suorituksen aikana.
3. Luokan runko suoritetaan kerran kun tulkki käsittelee luokan määritelmän. __init__ suoritetaan aina kun luokasta luodaan uusi olio.